<span type="title">Maven Basic</span> | <span type="update">2018-10-26</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 Maven 的使用，包括自动化框架的概念、Maven 的工程结构、常见的仓库使用、MVN 命令、Project Object Model 的概念以及其必要性，依赖的配置，传递，排除，Maven 插件和生命周期等。此外，还介绍了部署 Nexus 私有 Mavne 服务器，并且部署本地程序到 nexus 服务器的方法。</p></span>

# Maven 概述

## 自动化框架

Maven 是一个自动化的 Java 程序部署框架，其主要功能有：

1. 清理 清理原有的 classes 类目录
2. 编译 调用 JVM 编译 .java 程序
3. 测试 自动调用 JUnit 进行测试
4. 报告 报告编译和测试结果
5. 打包 将编译后的结果打成 war 或者 jar 包
6. 安装 Maven 特定概念，将程序依赖部署到合适的位置
7. 部署 提交容器进行部署

由于众所周知的原因：Java 的类名称、方法签名一般较长，语法更强调标准及其实现，而更少关注灵活性和便捷性，因此，常见的 Java 开发一般很难遇到使用 Code、Vim 等进行编写的。因此，在基于 IDE 的 Java 编程环境下，Maven 作为 “部署工具” 的属性难以体现，这是因为 Idea 或者 Elipse 等 IDE 基本上已经自动化的完成了 清理、编译、测试、打包的功能。

虽然 Maven 和大部分 IDE 的功能重叠，但是，大部分的 IDE 还都是提供了对于 Maven 较为全面的支持，在部署、包管理方面，Maven 的优势更大，尤其是 Java EE 开发环境下，SSH 框架至少有 20 个左右互相依赖的 jar 包，此外还有繁琐的版本问题、线上线下替换问题。因此，使用 Maven 管理包是一种主流的做法，况且，Maven [中心仓库](https://mvnrepository.com/)提供了大量的 Java jar 包，目前为止，大约有 1.2 亿个包。这些包的数量甚至超过除了 npm 以外的所有主流编程语言的主流仓库包数量的总和。

## Maven 工程结构

Maven 的工程结构如下：

```
根目录 工程名
 |-- pom.xml 核心配置文件
 |-- src/ 可读程序
 |     |-- main/ 主程序
 |     |    |-- java/ Java 源文件
 |     |    |-- resources/ 各种配置文件
 |     |-- test/ 测试程序
 |-- target/ class 源码
```

这种目录结构是一种约定俗称的规范，其中 src 即源码，包含了 main 主程序以及 test 测试程序主文件夹，target 即部署的二进制代码，pom.xml 则提供了 Maven 工程插件以及 jar 包依赖设置。

## Maven 仓库

仓库分为 中央仓库、中央仓库镜像、私服 Nexus、本地仓库。仓库中保存的内容是Maven 工程，包括 Maven 需要的插件，第三方框架或者工具的 jar 包，自有的 Maven 工程。一般常见的是中央仓库，除了中央仓库和其镜像，我们在本地文件夹还有一个本地缓存仓库，此外，为了全公司代码共享，还可以搭建 Nexus 私有仓库。私有仓库搭建见 https://help.sonatype.com/repomanager3/quick-start-guide---proxying-maven-and-npm 。

Maven 中心仓库地址是：https://mvnrepository.com/ 。其在全球各大洲都有广泛的镜像站点以供数据存取。想要获取一个 jar 包，只用到网站搜索，然后复制一段依赖信息，粘贴到 pom.xml 的依赖标签中即可，idea 等 IDE 会自动去镜像站下载，并且缓存在本地指定文件夹。

为了加速访问中心仓库，在 Maven 的 settings.xml 中 mirrors 标签中添加 aliyun 的镜像站点信息。

```
<mirror>
    <id>nexus-aliyun</id>
    <mirrorOf>central</mirrorOf>
    <name>Nexus aliyun</name>
    <url>http://maven.aliyun.com/nexus/content/groups/public</url>
</mirror>
```

当一个程序需要使用 jar 包，其会先到本地缓存寻找，如果找不到，则自动去中心仓库寻找。所有下载会在 `用户home目录/.m2/repository` 下。可以在 settings.xml 文件手动配置。


Maven 作为单独的程序，可以单独去 [Apache Maven 网站下载](https://maven.apache.org/
)，idea 提供了内置的 Maven，因此可以直接使用，新建 Maven 工程，默认使用内置的 Maven 程序，以及其默认仓库缓存位置和默认配置。如果希望在命令行执行 mvn 命令，则需要添加 Maven 到环境变量，其中需要定义 JRE_HOME, JAVA_HOME, MAVEN_HOME, M2_HOME, 此外，需要将 `%MAVEN_HOME%\bin;` 添加到 PATH 变量中。

## Maven 命令

Maven 常见的命令有：

- mvn clean 清理 target 目录
- mvn compile 编译
- mvn test-compile 测试编译
- mvn package 打包程序
- mvn install 安装此 jar 包
- mvn site 生成站点

需要注意，mvn 命令需要在 Maven 项目根目录下执行，也就是需要在同目录下有 pom.xml 文件。当执行命令的时候，默认会检查本地缓存是否有插件，如果没有，则自动去中心仓库下载。

执行 mvn 命令，会自动根据约定好的 maven 工程目录结构执行清理、编译、测试、打包等工作。比如，执行 clean 命令，自动删除 target 目录，执行 package 自动进行 clean、compile、test-compile、package，然后生成 jar 包在 target 目录下。命令有先后顺序，并且越靠近后面的命令会自动包含前面的命令，因此，如果需要打包，则自动会进行编译和测试，在没有问题后才打包，而不用按照流程逐个执行命令。

# Project Object Model

Maven 的每个工程特有的信息存放在 pom.xml 中，共有的信息存放在 settings.xml 中。其中 pom 的含义是 Project Object Model。具体而言，指的就是它的目录结构。

使用 groupid, artifactid, version 来定位一个 Maven 工程。其中， group id 由项目名称.公司域名倒序组成，artifact id 模块名称，version 版本信息

```
<groupid>com.mazhangjing.HelloAuto</groupid>
<artifactid>ModelA</artifactid>
<version>1.0.0</version>
```

上述工程对应地址为： `com/mazhangjing/HelloAuto/ModelA/1.0.0/ModelA-1.0.0.jar`

## Why Maven ?

这种方式和 Java 的 package 毫无关系，groupid 不一定要对应一个 package，artifactid 也不一定要对应此 package 的一个子 package。之所以没有使用原生 Java 的方式的原因是，原生的 Pacakge 难以管理（虽然相比较C，Python之流算得上是最好的程序分离解决方案），尤其是在 Java ee 环境中。在很多情况下，我们可能在不同工程中包含了相同类的不同代码（相同的类路径，但是存放了不同的程序）。因此，如果知道在某个 package 下存在某个类，但是这个类有可能在100个包含相同 package 的不同 jar 包中。此种情况尤其难以管理。

采用 group - artifact - version 的方式，只用在 pom 声明此依赖，IDE 会自动将其添加到类路径下，之后 Java 会自动扫描到此包的所有 jar 中的类，这就非常的方便，避免了对相同 package 的不同 jar 包的处理，现在只需要知道 artifactid 即可，也就是程序打包时的名称。

换句话说， Maven 的 POM 提供了一种独立于 Java Package 的方式提供了包管理的方式，这种方式以 jar 包为粒度进行划分，按照 jar 包的名称划分依赖。

虽然说， Java 的 Package 是一种非常理想化的，非常完美的中心主义的解决方案，设想一下，有一个巨大的仓库，存放了所有的 Java 程序，这些程序是按照 Package 进行划分的，查找和使用极其方便。

但是，现实情况是，我们不得不要在各处打包程序，因此不得不从这个仓库截取一小部分我们用到的目录结构，打成 jar 包，这就造成了严重的割裂——因为这些 jar 包的目录结构可能是重复的，jar 包没有任何，也没有能力去保证你需要的代码所依赖的 jar 包的目录结构不重合。这样的话，查找起来一个类所在的位置非常困难，并且难以管理。想象一下，有 100 个 jar 包，其中每个 jar 包包含 1000 个相同的目录结构，以及一个依赖这 1000 个目录结构的类，那么如果我们要使用一个 1000 个目录中的一个类， Java 需要查找 100 次来确定这个类的位置，这还没有算上这 100 个 jar 包共同的依赖的版本问题。

这也就是为什么，我们在下载各种开源 jar 包的时候，需要手动添加各种依赖，因为一旦哪个 jar 包试图包括其依赖的其余 jar 包，那么势必造成体积急剧膨胀，并且极其难以处理版本冲突，这将造成恶性循环 - 继续包含低版本的依赖程序，并且更加的难以修改和解除耦合。

Maven 提供的方案是，规定 jar 包的目录结构，要求其提供 pom 文件声明其对于其他 jar 包的依赖，这样的话，当我们声明了对于一个 jar 包的依赖，实际上就获取了此 jar 包和其依赖的 jar 包，没有任何冗余的目录结构，因此也更加方便管理版本冲突问题。

说白了，Maven 解决了一个什么问题呢？一句话概括就是：从无数个编译好的 jar 包中抽取其共同目录，然后以一种较细的粒度进行管理的问题。

要想完成这一功能，我们必须规定工程，或者说 jar 包的结构，包括 META-INF 和 pom.xml。

# Maven 配置

## 定义 Maven 工程

一个常见的 maven 工程的 pom.xml 如下所示：

```xml
<?xml version="1.0" encoding="UTF-8"?>
<project xmlns="http://maven.apache.org/POM/4.0.0"
         xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
         xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/xsd/maven-4.0.0.xsd">
    <modelVersion>4.0.0</modelVersion>

    <groupId>com.mazhangjing.hello</groupId>
    <artifactId>HelloAuto</artifactId>
    <version>1.0-RELEASE</version>
    <dependencies>
        <dependency>
            <groupId>junit</groupId>
            <artifactId>junit</artifactId>
            <version>RELEASE</version>
            <scope>test</scope>
        </dependency>
        <!-- https://mvnrepository.com/artifact/javax.servlet/javax.servlet-api -->
        <dependency>
            <groupId>javax.servlet</groupId>
            <artifactId>javax.servlet-api</artifactId>
            <version>4.0.1</version>
            <scope>provided</scope>
        </dependency>
        <!-- https://mvnrepository.com/artifact/javax.servlet.jsp/javax.servlet.jsp-api -->
        <dependency>
            <groupId>javax.servlet.jsp</groupId>
            <artifactId>javax.servlet.jsp-api</artifactId>
            <version>${cm_version}</version>
            <scope>provided</scope>
        </dependency>
        <!--jsp 等包很容易造成和服务器提供的 jar 包冲突问题，并且难以发现和解决，因此 scope 要慎重设置-->
    </dependencies>
    <properties>
        <cm_version>2.3.1</cm_version>
    </properties>
</project>
```

pom.xml 中包含了 groupid，artifactid，version，这三个属性可以定义一个 maven 工程。需要注意，习惯上而言，版本号后添加 RELEASE, SNAPSHOT 后缀表示释出版本或者快照版本，后者较不稳定（但是起码通过了测试）。此外，还有 LEAST 版本表示这是此版本号的最后一个版本(此版本号不再开发)。

## 依赖的添加和排除

maven 的依赖中，除了 groupid、artifactid 以及 version 以外，还有一个叫做 scope 的属性：

scope 的选项有： compile，test，provided。

- compile 表示主程序需要使用， 其对于主程序和测试程序都可见。参与打包。

- test 范围依赖 对于主程序不可见，只对 test 可见（因为只对test 有用）。如果在主程序强制引用 test 范围的包，则编译无法通过。不参与打包。

- provied 范围依赖，在开发的时候需要有，但是需要部署到服务器的时候不需要有（服务器提供，比如 TOMCAT Servlet）的包。对于主程序、测试程序均有效，不参与打包。

我们可以统一管理 XML 文件中的字段，比如版本信息，对于 Spring 框架而言，最好所有的组件都采用同一版本，因此，提供 properties 标签的子标签，然后定义此变量的值，之后使用 `${var}` 来使用此变量。
    
除了声明变量，还可以排除依赖的某个依赖。采用以下代码排除某个依赖中的某个依赖的依赖：

```xml
<exclusions>
    <exclusion>
        <g...>
        <a...>
        <v...>
        <s...>
    </exclusion>
</exclusions>
```
    
## 依赖的传递和优先级

当调用一个包，而这个包依赖其它包，那么调用此包不用调用这个包所依赖的包，所有的包都会被添加到依赖关系。常常使用一个工程来管理所有的依赖关系。但是，需要注意，非 compile 不能传递，必须手动添加。

依赖具有优先级：

路径不同时，路径最短者有限，如果依赖两个包A,B，其中有一个包 B 也声明了依赖包 A, 但是版本较低，根据路径最短，使用自身依赖 A，高版本被选用。

路径相同时，先声明者优先（XML 文件顺序）。


## Maven 的生命周期

Maven 各个构建环节的执行顺序固定，不能打乱，内置的顺序称之为生命周期。

内置三套生命周期： Clean LifeCycle, Site LifeCycle, Default LifeCycle

cl: pre-clean 在 clean 前完成的工作； clean 移除上一次构建文件；post-clean clean 后完成的工作

sl: pre-site 在生成站点文档前完成工作； site 生成文档； post-site 生成后的工作；site-deploy 文档部署到服务器

dl: 编译、测试、打包、构建

每个命令都包含了一系列的 plugin，如果要打包，直接 mvn package, 会自动调用 clean-plugin, compile-plugin... etc，然后再打包。

每个不同命令都包含了一些用于不同功能的 plugin，直接调用对应命令即可，不需要按照顺序执行命令。比如：

compile 生命周期 --- compile 插件目标 ---- maven-compiler-plugin 插件
test-compile 生命周期 ---- testCompile 插件目标 --- maven-compiler-plugin 插件

目标可以看作调用插件功能的命令。使用不同/相同插件的不同命令执行不同的生命周期。

# 私有 Maven 服务器：Nexus

Nexus 是一个基于 jetty 的私有 Maven 服务器产品。部署服务器需要 Oracle Java 8（不能是 OpenJDK）。

## 创建专有用户

为了方便管理，首先创建一个专有用户：

```
root@worker:/home# useradd -r -m -s /bin/bash spark 
root@worker:/home# passwd spark 

root@worker:/home#chmod +w /etc/sudoers 
root@worker:/home# vim /etc/sudoers
```

## 配置和绑定 JDK

安装 JDK8 的方法如下：

1.打开终端并运行命令添加PPA：

sudo add-apt-repository ppa:webupd8team/java

2.然后运行命令安装Java 8安装程序并在提示时接受许可证：

sudo apt-get install oracle-java8-installer

sudo apt-get install oracle-java8-set-default

卸载：移除PPA软件包总是很容易，只需打开终端并运行命令即可：

sudo apt-get remove --autoremove oracle-java8-installer oracle-java10-installer

安装后，下载 nexus 3 ，tar xvfz 解压后，到 bin 目录下，修改 nexus 文件，更改 `INSTALL4J_JAVA_HOME_OVERRIDE=/usr/lib/jvm/java-8-oracle`(此处的地址为你的 JDK 目录） 以完成 JDK 和 nexus 的绑定。

## 部署 nexus

直接运行 nexus 脚本即可：`./nexus run` 当出现 `Started Sonatype Nexus` 后表明服务器已经部署完毕。

按下 ctrl + c，退出服务，进入 `../sonatype-work/nexus3`，这里是用户的 data 默认目录，找到 `$data_dir/etc/nexus.properties` 文件，修改 `application-port=2333` 即可修改服务的默认监听端口。

## 配置 Maven 程序

当部署好之后，可以通过 web 访问界面，默认 admin/admin123 是管理员的用户名和密码，修改后，配置我们的私有服务器的仓库，一般而言，我们可以添加 aliyun 的中央镜像到私服，然后对于本地 maven，只提供私服接口，如果在私服找不到，自动去远程 proxy 的中央镜像服务器寻找，然后缓存在私有服务器，这里的概念就像 maven 本地服务器一样。

在 maven 的 setting.xml 中添加我们的仓库，启用认证即可：

```xml
<settings xmlns="http://maven.apache.org/SETTINGS/1.0.0"
      xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
      xsi:schemaLocation="http://maven.apache.org/SETTINGS/1.0.0
                          https://maven.apache.org/xsd/settings-1.0.0.xsd">
<profiles>
    <profile>  
      <id>jdk-1.8</id>  
      <activation>  
          <activeByDefault>true</activeByDefault>  
          <jdk>1.8</jdk>  
      </activation>  
      <properties>  
          <maven.compiler.source>1.8</maven.compiler.source>  
          <maven.compiler.target>1.8</maven.compiler.target>  
          <maven.compiler.compilerVersion>1.8</maven.compiler.compilerVersion>  
      </properties>   
    </profile>
</profiles>

<servers>
    <server>
          <id>cm_all</id>
          <username>corkine</username>
          <password>passwd</password>
    </server>
    <server>
          <id>maven-releases</id>
          <username>corkine</username>
          <password>passwd</password>
    </server>
    <server>
          <id>maven-snapshots</id>
          <username>corkine</username>
          <password>passwd</password>
    </server>
</servers>

<mirrors>
    <mirror>
      <id>cm_all</id>
      <url>http://yourhost.com/repository/cm-all/</url>
      <mirrorOf>central</mirrorOf>
    </mirror>
</mirrors>

</settings>
```

注意，这里只用配置私服（mirros标签）即可，可以为不同的私服仓库提供不同的登录用户名和密码（server标签），其中的 id 要对应私服的仓库名称。此外，为了避免各种版本问题，推荐设置 activation 和 properties 标签定义编译的JDK级别。

## Maven 的远程部署

为了部署到私有服务器，应该在每个工程中添加部署配置信息，其会自动使用 settings.xml 中的配置来使用凭证进行上传。
    
在 pom.xml 添加如下部署代码，即可将 releases 或者 snapshort 部署到指定的远程 nexus 仓库（使用 idea deploy 插件部署即可）。
    
```xml
<distributionManagement>
    <repository>
        <name>maven-releases</name>
        <id>maven-releases</id>
        <url>http://yourhost.com/repository/maven-releases/</url>
    </repository>
    <snapshotRepository>
        <name>maven-snapshots</name>
        <id>maven-snapshots</id>
        <url>http://yourhost.com/repository/maven-snapshots/</url>
    </snapshotRepository>
</distributionManagement>
```